# การใช้งาน Semantic Kernel กับการเชื่อมต่อ OpenBnB MCP Server

สมุดบันทึกนี้แสดงวิธีการใช้ Semantic Kernel ร่วมกับ OpenBnB MCP Server จริง เพื่อค้นหาที่พัก Airbnb จริงโดยใช้ MCPStdioPlugin สำหรับการเข้าถึง LLM จะใช้ Azure AI Foundry ในการตั้งค่าตัวแปรสภาพแวดล้อมของคุณ คุณสามารถทำตาม [บทเรียนการตั้งค่า](/00-course-setup/README.md)


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio

from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## การสร้างการเชื่อมต่อ MCP Plugin

เราจะเชื่อมต่อกับ [เซิร์ฟเวอร์ OpenBnB MCP](https://github.com/openbnb-org/mcp-server-airbnb) โดยใช้ MCPStdioPlugin เซิร์ฟเวอร์นี้ให้บริการฟังก์ชันการค้นหา Airbnb ผ่านแพ็กเกจ @openbnb/mcp-server-airbnb


## การสร้าง Client

ในตัวอย่างนี้ เราจะใช้ Azure AI Foundry สำหรับการเข้าถึง LLM ของเรา อย่าลืมตั้งค่าตัวแปรสภาพแวดล้อมให้ถูกต้อง


## การตั้งค่าสภาพแวดล้อม

ตั้งค่าการใช้งาน Azure OpenAI ตรวจสอบให้แน่ใจว่าคุณได้ตั้งค่าตัวแปรสภาพแวดล้อมดังต่อไปนี้:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## การทำความเข้าใจการเชื่อมต่อ OpenBnB MCP

โน้ตบุ๊กนี้เชื่อมต่อกับ **เซิร์ฟเวอร์ OpenBnB MCP จริง** ที่ให้บริการฟังก์ชันการค้นหา Airbnb แบบจริง

### วิธีการทำงาน:

1. **MCPStdioPlugin**: ใช้การสื่อสารผ่าน input/output มาตรฐานกับเซิร์ฟเวอร์ MCP
2. **แพ็กเกจ NPM จริง**: ดาวน์โหลดและรัน `@openbnb/mcp-server-airbnb` ผ่าน npx
3. **ข้อมูลสด**: ส่งคืนข้อมูลอสังหาริมทรัพย์ Airbnb จริงจาก API ของพวกเขา
4. **การค้นพบฟังก์ชัน**: ตัวแทนจะค้นพบฟังก์ชันที่มีอยู่จากเซิร์ฟเวอร์ MCP โดยอัตโนมัติ

### ฟังก์ชันที่มีอยู่:

เซิร์ฟเวอร์ OpenBnB MCP มักจะให้บริการ:
- **search_listings** - ค้นหาอสังหาริมทรัพย์ Airbnb ตามสถานที่และเงื่อนไข
- **get_listing_details** - รับข้อมูลรายละเอียดเกี่ยวกับอสังหาริมทรัพย์เฉพาะ
- **check_availability** - ตรวจสอบความพร้อมใช้งานสำหรับวันที่เฉพาะ
- **get_reviews** - ดึงรีวิวสำหรับอสังหาริมทรัพย์
- **get_host_info** - รับข้อมูลเกี่ยวกับเจ้าของอสังหาริมทรัพย์

### สิ่งที่ต้องเตรียม:

- **Node.js** ติดตั้งในระบบของคุณ
- **การเชื่อมต่ออินเทอร์เน็ต** เพื่อดาวน์โหลดแพ็กเกจเซิร์ฟเวอร์ MCP
- **NPX** พร้อมใช้งาน (มาพร้อมกับ Node.js)

### การทดสอบการเชื่อมต่อ:

คุณสามารถทดสอบเซิร์ฟเวอร์ MCP ด้วยตนเองโดยรัน:
```bash
npx -y @openbnb/mcp-server-airbnb
```

สิ่งนี้จะดาวน์โหลดและเริ่มต้นเซิร์ฟเวอร์ OpenBnB MCP ซึ่ง Semantic Kernel จะเชื่อมต่อเพื่อดึงข้อมูล Airbnb จริง


## การรัน Agent กับ OpenBnB MCP Server

ตอนนี้เราจะรัน AI Agent ที่เชื่อมต่อกับ OpenBnB MCP server เพื่อค้นหาที่พัก Airbnb จริงในสตอกโฮล์มสำหรับผู้ใหญ่ 2 คนและเด็ก 1 คน คุณสามารถเปลี่ยนรายการ `user_inputs` เพื่อปรับเปลี่ยนเกณฑ์การค้นหาได้ตามต้องการ


In [ ]:
# Main execution cell - Enhanced with proper HTML rendering and MCP tool logging
# User requests for Airbnb search
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        # Create MCP plugin connection to real OpenBnB server
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb", "--ignore-robots-txt"],
        ) as airbnb_plugin:

            print("🔧 MCP Plugin created and connected")

            # Load tools for function discovery
            await airbnb_plugin.load_tools()
            await asyncio.sleep(3)  # Give more time for initialization
            print("✅ Tools loaded from MCP server")

            # Debug: Check what tools were loaded
            if hasattr(airbnb_plugin, '_tools'):
                print(f"📋 Internal tools: {airbnb_plugin._tools}")

            # Verify available functions
            funcs = [attr for attr in dir(airbnb_plugin)
                     if callable(getattr(airbnb_plugin, attr))
                     and attr in ['airbnb_search', 'airbnb_listing_details']]
            print(f"📋 Available functions: {funcs}")

            # Create agent with Azure OpenAI service
            agent = ChatCompletionAgent(
                service=AzureChatCompletion(),  # Use default constructor
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the airbnb_search function to find properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("🤖 Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 Processing request: {user_input}")
                
                # Track MCP tool usage
                mcp_tools_used = []
                function_calls_log = []
                
                # Try streaming to capture function calls
                try:
                    agent_name = None
                    full_response = []
                    current_function_name = None
                    argument_buffer = ""
                    
                    async for response in agent.invoke_stream(
                        messages=user_input,
                        thread=thread,
                    ):
                        thread = response.thread
                        agent_name = response.name
                        
                        for item in response.items:
                            # Log function calls
                            if isinstance(item, FunctionCallContent):
                                if item.function_name:
                                    current_function_name = item.function_name
                                    mcp_tools_used.append(item.function_name)
                                    print(f"\n🔧 MCP Tool Selected: {item.function_name}")
                                    
                                if isinstance(item.arguments, str):
                                    argument_buffer += item.arguments
                            
                            # Log function results
                            elif isinstance(item, FunctionResultContent):
                                if current_function_name:
                                    try:
                                        args = json.loads(argument_buffer.strip()) if argument_buffer else {}
                                    except:
                                        args = {"raw": argument_buffer}
                                    
                                    function_calls_log.append({
                                        "function": current_function_name,
                                        "arguments": args,
                                        "timestamp": asyncio.get_event_loop().time()
                                    })
                                    
                                    print(f"   📍 Arguments: {json.dumps(args, indent=2)}")
                                    print(f"   ✅ MCP Tool Executed Successfully")
                                    
                                    current_function_name = None
                                    argument_buffer = ""
                            
                            # Collect response text
                            elif isinstance(item, StreamingTextContent) and item.text:
                                full_response.append(item.text)
                    
                    # Join the full response
                    response_text = ''.join(full_response)
                    
                except Exception as e:
                    print(f"⚠️ Streaming failed, using get_response: {str(e)[:100]}")
                    # Fallback to non-streaming
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    response_text = str(response)
                    agent_name = response.name
                
                
                # Process the response to ensure HTML tables render correctly
                # Remove any markdown code blocks around HTML
                response_text = response_text.replace('```html', '').replace('```', '')
                
                # Ensure proper HTML structure for tables
                if '<table' in response_text.lower():
                    # Add CSS styling for better table rendering
                    table_css = """
                    <style>
                        .airbnb-results table {
                            border-collapse: collapse;
                            width: 100%;
                            margin: 10px 0;
                        }
                        .airbnb-results th, .airbnb-results td {
                            border: 1px solid #ddd;
                            padding: 8px;
                            text-align: left;
                        }
                        .airbnb-results th {
                            background-color: #f2f2f2;
                            font-weight: bold;
                        }
                        .airbnb-results tr:nth-child(even) {
                            background-color: #f9f9f9;
                        }
                        .airbnb-results a {
                            color: #1976d2;
                            text-decoration: none;
                        }
                        .airbnb-results a:hover {
                            text-decoration: underline;
                        }
                    </style>
                    """
                    response_text = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                
                # Build the complete HTML output
                html_output = f"""
                <div style='margin:10px; padding:10px; border-left:3px solid #2E8B57; background:#F0F8FF;'>
                    <strong>User:</strong> {user_input}
                </div>
                """
                
                # Add function call details if available
                if function_calls_log:
                    details_html = "<details style='margin:10px; padding:10px; background:#f5f5f5;'>"
                    details_html += "<summary><strong>📊 Function Call Details</strong></summary>"
                    details_html += "<pre style='background:#fff; padding:10px; overflow-x:auto;'>"
                    for call in function_calls_log:
                        details_html += f"Function: {call['function']}\n"
                        details_html += f"Arguments: {json.dumps(call['arguments'], indent=2)}\n"
                        details_html += "---\n"
                    details_html += "</pre></details>"
                    html_output += details_html
                
                # Add the agent's response with proper HTML rendering
                html_output += f"""
                <div style='margin:10px; padding:15px; border-left:3px solid #1E90FF; background:#FFFFFF;'>
                    <strong>{agent_name}:</strong><br>
                    {response_text}
                </div>
                """
                
                # Display the HTML with proper rendering
                display(HTML(html_output))
                
                
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        import traceback
        traceback.print_exc()

print("🚀 Starting with Azure OpenAI...")
await main()
print("✅ Done!")

สรุป  
ขอแสดงความยินดี! คุณได้สร้าง AI agent ที่สามารถเชื่อมต่อกับการค้นหาที่พักในโลกจริงผ่าน Model Context Protocol (MCP) ได้สำเร็จ:

เทคโนโลยีที่ใช้:  
- Semantic Kernel - สำหรับสร้าง intelligent agents ด้วย Azure OpenAI  
- Azure AI Foundry - สำหรับความสามารถของ LLM และการตอบกลับในรูปแบบแชท  
- MCP (Model Context Protocol) - สำหรับการรวมเครื่องมือในรูปแบบมาตรฐาน  
- OpenBnB MCP Server - สำหรับการค้นหาที่พัก Airbnb จริง  
- Node.js/NPX - สำหรับการรัน MCP server ภายนอก  

สิ่งที่คุณได้เรียนรู้:  
- การรวม MCP: การเชื่อมต่อ Semantic Kernel agents กับ MCP servers ภายนอก  
- การเข้าถึงข้อมูลแบบเรียลไทม์: การค้นหาที่พัก Airbnb จริงผ่าน API แบบสด  
- การสื่อสารผ่านโปรโตคอล: การใช้ stdio ในการสื่อสารระหว่าง agent และ MCP server  
- การค้นพบฟังก์ชัน: การค้นหาฟังก์ชันที่มีอยู่จาก MCP servers โดยอัตโนมัติ  
- การตอบกลับแบบสตรีม: การจับและบันทึกการเรียกฟังก์ชันแบบเรียลไทม์  
- การเรนเดอร์ HTML: การจัดรูปแบบการตอบกลับของ agent ด้วยตารางที่มีสไตล์และการแสดงผลแบบโต้ตอบ  

ขั้นตอนถัดไป:  
- รวม MCP servers เพิ่มเติม (เช่น สภาพอากาศ, เที่ยวบิน, ร้านอาหาร)  
- สร้างระบบ multi-agent ที่รวม MCP และ A2A protocols  
- สร้าง MCP servers แบบกำหนดเองสำหรับแหล่งข้อมูลของคุณเอง  
- เพิ่มความสามารถในการจดจำการสนทนาแบบต่อเนื่องระหว่างเซสชัน  
- นำ agent ไปใช้งานบน Azure Functions พร้อมการจัดการ MCP server  
- เพิ่มการยืนยันตัวตนของผู้ใช้และความสามารถในการจอง  

ข้อได้เปรียบหลักของสถาปัตยกรรม MCP:  
- มาตรฐาน: โปรโตคอลสากลสำหรับการเชื่อมต่อ AI agents กับเครื่องมือภายนอก  
- ข้อมูลแบบเรียลไทม์: การเข้าถึงข้อมูลสดและทันสมัยจากบริการต่าง ๆ  
- ความสามารถในการขยาย: การรวมแหล่งข้อมูลและเครื่องมือใหม่ได้ง่าย  
- การทำงานร่วมกัน: ใช้งานได้กับ AI frameworks และแพลตฟอร์ม agent ต่าง ๆ  
- การแยกหน้าที่: การแยกความแตกต่างระหว่างตรรกะของ AI และการเข้าถึงข้อมูลภายนอกอย่างชัดเจน  



---

**ข้อจำกัดความรับผิดชอบ**:  
เอกสารนี้ได้รับการแปลโดยใช้บริการแปลภาษา AI [Co-op Translator](https://github.com/Azure/co-op-translator) แม้ว่าเราจะพยายามให้การแปลมีความถูกต้อง แต่โปรดทราบว่าการแปลอัตโนมัติอาจมีข้อผิดพลาดหรือความไม่ถูกต้อง เอกสารต้นฉบับในภาษาดั้งเดิมควรถือเป็นแหล่งข้อมูลที่เชื่อถือได้ สำหรับข้อมูลที่สำคัญ ขอแนะนำให้ใช้บริการแปลภาษามืออาชีพ เราไม่รับผิดชอบต่อความเข้าใจผิดหรือการตีความผิดที่เกิดจากการใช้การแปลนี้
